## DATA AUGMENTED


In [1]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from zipfile import ZipFile
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

#JUPYTER VERSION
#comb_path = "../Data/Combined_Images"

#GOOGLE COLAB VERSION
from google.colab import drive
drive.mount('/content/drive')

# Unzipping the dataset file facial-age.zip

combined_images_path = "/content/drive/MyDrive/Data/Combined_Images.zip"

with ZipFile(combined_images_path, 'r') as myzip:
    myzip.extractall("../content/Combined_Images")
    print('Done unzipping Combined_Images.zip')

comb_path = '../content/Combined_Images' 
batch_size = 64

train_ds = tf.keras.utils.image_dataset_from_directory(
  comb_path,
  validation_split=0.2,
  subset="training", #If should be return the training set (80%) or the validation set (20%)
  seed=41, #Seed should guarantee that train_ds and val_ds doesn't have common images
  shuffle=True,
  image_size=(200, 200),
  batch_size=batch_size,
  color_mode='grayscale')

val_ds = tf.keras.utils.image_dataset_from_directory(
  comb_path,
  validation_split=0.2,
  subset="validation",
  seed=41, 
  shuffle=True,
  image_size=(200, 200),
  batch_size=batch_size,
  color_mode='grayscale')

test_dataset = val_ds.take(53)
val_ds = val_ds.skip(53)

print('Batches for training -->', train_ds.cardinality())
print('Batches for validating -->', val_ds.cardinality())
print('Batches for testing -->', test_dataset.cardinality())    

Mounted at /content/drive
Done unzipping Combined_Images.zip
Found 33884 files belonging to 8 classes.
Using 27108 files for training.
Found 33884 files belonging to 8 classes.
Using 6776 files for validation.
Batches for training --> tf.Tensor(424, shape=(), dtype=int64)
Batches for validating --> tf.Tensor(53, shape=(), dtype=int64)
Batches for testing --> tf.Tensor(53, shape=(), dtype=int64)


In [2]:
data_augmentation = tf.keras.Sequential([
  #tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomHeight(0.3),
  tf.keras.layers.RandomWidth(0.3),
  #tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.RandomZoom(0.3),
  tf.keras.layers.RandomContrast(0.2),
  #tf.keras.layers.RandomBrightness([-0.8,0.8]),  #Need Tensorflow 2.5.0
  tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2)
])

#resize = tf.keras.layers.Resizing(out_height, out_width)
#height = tf.keras.layers.RandomHeight(0.3)
#width = tf.keras.layers.RandomWidth(0.3)
#zoom = tf.keras.layers.RandomZoom(0.3)
#flip = tf.keras.layers.RandomFlip("horizontal_and_vertical") # or "horizontal", "vertical"
#rotate = tf.keras.layers.RandomRotation(0.2)
#crop = tf.keras.layers.RandomCrop(out_height, out_width)
#translation = tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2)
#brightness = tf.keras.layers.RandomBrightness([-0.8,0.8])
#contrast = tf.keras.layers.RandomContrast(0.2)

In [3]:
#RandomHeight and RadomWidth will lead to a None shape on the height dimension, as not all outputs from the layer will be the same height (by design). That is ok for things like the 
#Conv2D layer, which can accept variable shaped image input (with None shapes on some dimensions).
#This will not work for then calling into a Flatten followed by a Dense, because the flattened batches will also be of variable size (because of the variable height), and the 
#Dense layer needs a fixed shape for the last dimension. You could probably pad output of flatten before the dense, but if you want this architecture, you may just want to 
#avoid image augmentation layer that lead to a variable output shape.

#So instead of using a Flatten layer, you could, for example, use a GlobalMaxPool2D layer, which does not need to know the other dimensions beforehand

model4 = tf.keras.Sequential([
  data_augmentation,
  tf.keras.layers.Rescaling(1./255, input_shape=(200, 200, 1)),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu', kernel_regularizer=regularizers.l2(l=0.01)),
  tf.keras.layers.MaxPooling2D(),
  #tf.keras.layers.Flatten(), #See reference above
  tf.keras.layers.GlobalMaxPool2D(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(8, activation='softmax')
])

#model.compile(optimizer=tf.keras.optimizers.Adam(),
              #loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              #metrics=tf.keras.metrics.Accuracy())

#Additionaly, if you do not one-hot encode your data, set sparse_categorical_crossentropy as loss and sparse_categorical_accuracy as metric.
model4.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['sparse_categorical_accuracy'])

# Defining the early stop to monitor the validation loss to avoid overfitting.
early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')

epochs=80
history = model4.fit(
  train_ds,
  validation_data=val_ds,
  callbacks=[early_stop],
  epochs=epochs,
  shuffle=True
)

Epoch 1/80
424/424 [==============================] - 94s 189ms/step - loss: 2.1224 - sparse_categorical_accuracy: 0.1466 - val_loss: 2.0380 - val_sparse_categorical_accuracy: 0.1460
Epoch 2/80
424/424 [==============================] - 73s 171ms/step - loss: 2.0302 - sparse_categorical_accuracy: 0.1503 - val_loss: 2.0288 - val_sparse_categorical_accuracy: 0.1504
Epoch 3/80
424/424 [==============================] - 68s 160ms/step - loss: 2.0205 - sparse_categorical_accuracy: 0.1729 - val_loss: 2.0180 - val_sparse_categorical_accuracy: 0.1983
Epoch 4/80
424/424 [==============================] - 65s 153ms/step - loss: 1.9881 - sparse_categorical_accuracy: 0.2053 - val_loss: 1.9999 - val_sparse_categorical_accuracy: 0.1871
Epoch 5/80
424/424 [==============================] - 59s 137ms/step - loss: 1.9793 - sparse_categorical_accuracy: 0.2093 - val_loss: 2.0091 - val_sparse_categorical_accuracy: 0.1826
Epoch 6/80
424/424 [==============================] - 58s 136ms/step - loss: 1.9726 -

KeyboardInterrupt: ignored